In [ ]:
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##To determine the evaluation metrics for the detections which are observed by YOLOv5 and Faster R-CNN

In [ ]:
import os
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as T
import torch.nn.functional as F

# Define the path to the folder containing the images
folder_path = '/content/drive/MyDrive/Smart-Beach-Predict-People/test-data'

# Create the "output" folder if it doesn't exist
output_folder = '/content/drive/MyDrive/Smart-Beach-Predict-People/test-data/op'
os.makedirs(output_folder, exist_ok=True)

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)
yolo_model.eval()

# Load Faster R-CNN model
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
frcnn_model.eval()

# Confidence threshold for YOLOv5
yolo_confidence_threshold = 0.7

# Confidence threshold for Faster R-CNN
frcnn_confidence_threshold = 0.7

# Function to divide the image into 4 zones using one horizontal and one vertical line
def divide_into_zones(image):
    width, height = image.size
    # Define coordinates for the vertical and horizontal lines
    vertical_line = width // 2
    horizontal_line = height // 2

    # Define the zones
    zones = [
        (0, 0, vertical_line, horizontal_line),                   # Zone 1
        (vertical_line, 0, width, horizontal_line),              # Zone 2
        (0, horizontal_line, vertical_line, height),             # Zone 3
        (vertical_line, horizontal_line, width, height)          # Zone 4
    ]

    return zones

# Function to perform object detection using YOLOv5
def detect_objects_yolo(image):
    results = yolo_model(image)
    bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
    class_indices = results.xyxy[0][:, 5].cpu().numpy().astype(int)  # Class indices
    class_names = [results.names[idx] for idx in class_indices]  # Class names
    return bboxes, class_names


# Function to perform object detection using Faster R-CNN
def detect_objects_frcnn(image):
    image_tensor = T.ToTensor()(image)
    with torch.no_grad():
        prediction = frcnn_model([image_tensor])[0]
    bboxes = prediction['boxes'].cpu().numpy()
    labels = prediction['labels'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()
    class_names = [str(i) for i in labels]  # Dummy class names as we're not using actual class names here
    return bboxes, class_names

# Function to perform ensemble voting
def ensemble_voting(images):
    bboxes_yolo = []
    class_names_yolo = []
    bboxes_frcnn = []
    labels_frcnn = []

    for image in images:
        bboxes_y, class_names_y = detect_objects_yolo(image)
        bboxes_yolo.append(bboxes_y)
        class_names_yolo.append(class_names_y)

        bboxes_f, class_names_f = detect_objects_frcnn(image)
        bboxes_frcnn.append(bboxes_f)
        labels_frcnn.append(class_names_f)

    return bboxes_yolo, class_names_yolo, bboxes_frcnn, labels_frcnn

# Function to compute IoU between two bounding boxes
def compute_iou(box1, box2):
    # Compute intersection coordinates
    x1_intersection = max(box1[0], box2[0])
    y1_intersection = max(box1[1], box2[1])
    x2_intersection = min(box1[2], box2[2])
    y2_intersection = min(box1[3], box2[3])

    # Compute intersection area
    intersection_area = max(0, x2_intersection - x1_intersection + 1) * max(0, y2_intersection - y1_intersection + 1)

    # Compute union area
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = box1_area + box2_area - intersection_area

    # Compute IoU
    iou = intersection_area / union_area
    return iou

# Function to evaluate precision, recall, and accuracy
def evaluate_performance(gt_bboxes, pred_bboxes, iou_threshold=0.5):
    num_gt = len(gt_bboxes)
    num_pred = len(pred_bboxes)

    # Initialize lists to store matched ground truth and predicted indices
    matched_gt_indices = []
    matched_pred_indices = []

    # Match predicted bounding boxes to ground truth boxes based on IoU
    for i in range(num_gt):
        for j in range(num_pred):
            iou = compute_iou(gt_bboxes[i], pred_bboxes[j])
            # If the IoU is above the threshold and the ground truth box is not already matched
            if iou >= iou_threshold and i not in matched_gt_indices and j not in matched_pred_indices:
                matched_gt_indices.append(i)
                matched_pred_indices.append(j)

    # Calculate precision, recall, and accuracy
    precision = len(matched_gt_indices) / num_pred if num_pred > 0 else 0
    recall = len(matched_gt_indices) / num_gt if num_gt > 0 else 0
    accuracy = len(matched_gt_indices) / num_gt if num_gt > 0 else 0

    return precision, recall, accuracy

# Function to classify objects and count their occurrences within each zone
def classify_and_count_objects(image, zones):
    # Perform ensemble voting
    bboxes_yolo, _, bboxes_frcnn, _ = ensemble_voting([image])

    # Convert bounding boxes to list of tuples
    bboxes_yolo = [(bbox[0], bbox[1], bbox[2], bbox[3]) for bbox in bboxes_yolo[0]]
    bboxes_frcnn = [(bbox[0], bbox[1], bbox[2], bbox[3]) for bbox in bboxes_frcnn[0]]

    # Evaluate performance
    precision, recall, accuracy = evaluate_performance(bboxes_yolo, bboxes_frcnn)

    # Process YOLOv5 results
    zone_counts = {zone_id: {} for zone_id in range(len(zones))}
    for bbox in bboxes_yolo:
        for zone_id, zone_coords in enumerate(zones):
            x1, y1, x2, y2 = bbox
            if zone_coords[0] <= x1 <= zone_coords[2] and zone_coords[1] <= y1 <= zone_coords[3]:
                if 'Object' in zone_counts[zone_id]:
                    zone_counts[zone_id]['Object'] += 1
                else:
                    zone_counts[zone_id]['Object'] = 1
                break

    return zone_counts, precision, recall, accuracy

# Iterate over each image file
for image_file in os.listdir(folder_path):
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)

    # Skip directories
    if os.path.isdir(image_path):
        continue

    try:
        # Load the original image
        original_image = Image.open(image_path)
    except UnidentifiedImageError:
        print(f"Error: Unable to identify image file '{image_path}'. Skipping.")
        continue

    # Divide the image into 4 zones
    zones = divide_into_zones(original_image)

    # Classify objects, count their occurrences within each zone, and evaluate performance
    zone_counts, precision, recall, accuracy = classify_and_count_objects(original_image, zones)

    # Create a figure and axis for plotting
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image with the 4 zones
    axs[0].imshow(original_image)
    for zone_id, zone_coords in enumerate(zones):
        rect = plt.Rectangle((zone_coords[0], zone_coords[1]),
                             zone_coords[2] - zone_coords[0],
                             zone_coords[3] - zone_coords[1],
                             fill=False, edgecolor='red', linewidth=2)
        axs[0].add_patch(rect)
        axs[0].text(zone_coords[0] + 5, zone_coords[1] + 20, f'Zone {zone_id + 1}',
                    color='red', fontsize=12, weight='bold')

    # Plot the count information beside the image
    axs[1].axis('off')
    for zone_id, zone_counts_dict in zone_counts.items():
        zone_str = f'Zone {zone_id + 1} Counts:\n'
        count_str = '\n'.join([f'- {class_name}: {count}' for class_name, count in zone_counts_dict.items()])
        axs[1].text(0.5, 0.95 - 0.25 * zone_id, zone_str + count_str,
                    transform=axs[1].transAxes, fontsize=12, weight='bold')

    # Adjust layout
    plt.tight_layout()

    # Save the modified image with both the original image and counts
    output_path = os.path.join(output_folder, f"output_{image_file}")
    plt.savefig(output_path)
    plt.close()

    print(f"Processed image with counts saved at: {output_path}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")


## To detect the object counts using the image classification of Yolov5 and Faster R-CNN models

In [ ]:
import os
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as T
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support

# Define the path to the folder containing the images
folder_path = '/content/drive/MyDrive/Smart-Beach-Predict-People/test-data'

# Create the "output" folder if it doesn't exist
output_folder = '/content/drive/MyDrive/Smart-Beach-Predict-People/test-data/op'
os.makedirs(output_folder, exist_ok=True)

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)
yolo_model.eval()

# Load Faster R-CNN model
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
frcnn_model.eval()

# Confidence threshold for YOLOv5
yolo_confidence_threshold = 0.2

# Confidence threshold for Faster R-CNN
frcnn_confidence_threshold = 0.5

# Function to divide the image into 4 zones using one horizontal and one vertical line
def divide_into_zones(image):
    width, height = image.size
    # Define coordinates for the vertical and horizontal lines
    vertical_line = width // 2
    horizontal_line = height // 2

    # Define the zones
    zones = [
        (0, 0, vertical_line, horizontal_line),                   # Zone 1
        (vertical_line, 0, width, horizontal_line),              # Zone 2
        (0, horizontal_line, vertical_line, height),             # Zone 3
        (vertical_line, horizontal_line, width, height)          # Zone 4
    ]

    return zones

# Function to perform object detection using YOLOv5
def detect_objects_yolo(image):
    results = yolo_model(image)
    bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
    class_indices = results.xyxy[0][:, 5].cpu().numpy().astype(int)  # Class indices
    class_names = [results.names[idx] for idx in class_indices]  # Class names
    return bboxes, class_names


# Function to perform object detection using Faster R-CNN
def detect_objects_frcnn(image):
    image_tensor = T.ToTensor()(image)
    with torch.no_grad():
        prediction = frcnn_model([image_tensor])[0]
    bboxes = prediction['boxes'].cpu().numpy()
    labels = prediction['labels'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()
    class_names = [str(i) for i in labels]  # Dummy class names as we're not using actual class names here
    return bboxes, class_names

# Function to perform ensemble voting
def ensemble_voting(images):
    bboxes_yolo = []
    class_names_yolo = []
    bboxes_frcnn = []
    labels_frcnn = []

    for image in images:
        bboxes_y, class_names_y = detect_objects_yolo(image)
        bboxes_yolo.append(bboxes_y)
        class_names_yolo.append(class_names_y)

        bboxes_f, class_names_f = detect_objects_frcnn(image)
        bboxes_frcnn.append(bboxes_f)
        labels_frcnn.append(class_names_f)

    return bboxes_yolo, class_names_yolo, bboxes_frcnn, labels_frcnn

# Function to classify objects and count their occurrences within each zone
def classify_and_count_objects(image, zones):
    # Perform ensemble voting
    bboxes_yolo, class_names_yolo, bboxes_frcnn, labels_frcnn = ensemble_voting([image])

    # Process YOLOv5 results
    zone_counts = {zone_id: {} for zone_id in range(len(zones))}
    for bbox, class_name in zip(bboxes_yolo[0], class_names_yolo[0]):
        for zone_id, zone_coords in enumerate(zones):
            x1, y1, x2, y2 = bbox.astype(int)
            if zone_coords[0] <= x1 <= zone_coords[2] and zone_coords[1] <= y1 <= zone_coords[3]:
                if class_name in zone_counts[zone_id]:
                    zone_counts[zone_id][class_name] += 1
                else:
                    zone_counts[zone_id][class_name] = 1
                break

    return zone_counts

# Calculate evaluation metrics
def calculate_metrics(true_labels, pred_labels):
    precision, recall, fscore, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')
    return precision, recall, fscore

# Create lists to store true and predicted labels
true_labels_list = []
pred_labels_list = []

# Iterate over each image file
for image_file in os.listdir(folder_path):
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)

    # Skip directories
    if os.path.isdir(image_path):
        continue

    # Load the original image
    original_image = Image.open(image_path)

    # Divide the image into 4 zones
    zones = divide_into_zones(original_image)

    # Classify objects, count their occurrences within each zone, and create bounding boxes
    zone_counts = classify_and_count_objects(original_image, zones)

    # Extract true labels from the image file name (assuming file name contains true labels)
    true_labels = [label.split('_')[0] for label in image_file.split('.')[0].split('_')[1:]]

    # Extract predicted labels from the zone_counts dictionary
    pred_labels = []
    for zone_counts_dict in zone_counts.values():
        pred_labels.extend(list(zone_counts_dict.keys()))

    # Update true and predicted labels lists
    true_labels_list.extend(true_labels)
    pred_labels_list.extend(pred_labels)

    # Create a figure and axis for plotting
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image with the 4 zones
    axs[0].imshow(original_image)
    for zone_id, zone_coords in enumerate(zones):
        rect = plt.Rectangle((zone_coords[0], zone_coords[1]),
                             zone_coords[2] - zone_coords[0],
                             zone_coords[3] - zone_coords[1],
                             fill=False, edgecolor='red', linewidth=2)
        axs[0].add_patch(rect)
        axs[0].text(zone_coords[0] + 5, zone_coords[1] + 20, f'Zone {zone_id + 1}',
                    color='red', fontsize=12, weight='bold')

    # Plot the count information beside the image
    axs[1].axis('off')
    for zone_id, zone_counts_dict in zone_counts.items():
        zone_str = f'Zone {zone_id + 1} Counts:\n'
        count_str = '\n'.join([f'- {class_name}: {count}' for class_name, count in zone_counts_dict.items()])
        axs[1].text(0.5, 0.95 - 0.25 * zone_id, zone_str + count_str,
                    transform=axs[1].transAxes, fontsize=12, weight='bold')

    # Adjust layout
    plt.tight_layout()

    # Save the modified image with both the original image and counts
    output_path = os.path.join(output_folder, f"output1_{image_file}")
    plt.savefig(output_path)
    plt.close()

    print(f"Processed image with counts saved at: {output_path}")

